In [ ]:
from collections import OrderedDict
from glob import glob
import numpy as np
import pandas as pd
from os.path import join
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import xarray as xr

import torch
import torch.nn as nn
from torch import Tensor
from torchmetrics.utilities.checks import _check_same_shape
from torchmetrics import Metric
import pytorch_lightning as pl
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, TensorDataset
import pylab as plt
import matplotlib

In [ ]:
import sys
sys.path.insert(0, "../pismemulator/")

from nnemulator import PISMDataset

In [ ]:
dataset = PISMDataset(data_dir="../data/speeds_v2/",
                      samples_file="../data/samples/velocity_calibration_samples_100.csv",
                      target_file="../data/observed_speeds/greenland_obs_g1800m.nc",
                     target_corr_threshold=25)
            

In [ ]:
cd ../speedemulator/

In [ ]:
run ../speedemulator/sample_posterior.py --thinning_factor 10 --emulator_dir 2021_11_gris_2km_cpu --data_dir ../data/speeds_v2/ --target_file ../data/observed_speeds/greenland_obs_g1800m.nc

In [ ]:
data_dir="../data/speeds_v2/"
samples_file="../data/samples/velocity_calibration_samples_100.csv"
target_file="../data/observed_speeds/greenland_obs_g1800m.nc"
target_var="velsurf_mag"
target_corr_threshold=25.0
target_corr_var="thickness"
target_error_var="velsurf_mag_error"
training_var="velsurf_mag"
thinning_factor=1
normalize_x=True
log_y=True
threshold=100e3
epsilon=0
return_numpy=False
